In [21]:
import os
print(os.environ["MINIO_ACCESS_KEY"])

mEk4ek2HCmeb3xCK2twM


In [22]:
from pyspark.sql import SparkSession

In [23]:
spark = (
        SparkSession.builder.master("spark://spark:7077")
        .appName("stg_clockify__time_entries")
        .config(
            "spark.jars.packages",
            "org.apache.hadoop:hadoop-aws:3.3.4,org.apache.iceberg:iceberg-spark-runtime-3.5_2.12:1.4.2",
        )
        .config("spark.hadoop.fs.s3a.access.key", os.environ["MINIO_ACCESS_KEY"])
        .config("spark.hadoop.fs.s3a.secret.key", os.environ["MINIO_SECRET_KEY"])
        .config("spark.hadoop.fs.s3a.endpoint", "http://minio:9000")
        .config("spark.hadoop.fs.s3a.path.style.access", "true")
        .config("spark.hadoop.fs.s3a.connection.ssl.enabled", "false")
        .config(
            "spark.sql.catalog.clockify_catalog",
            "org.apache.iceberg.spark.SparkCatalog",
        )
        .config("spark.sql.catalog.clockify_catalog.type", "hadoop")
        .config(
            "spark.sql.catalog.clockify_catalog.warehouse", f"s3a://staging/"
        )
        .getOrCreate()
    )

In [24]:
df = spark.read.format("iceberg").load("clockify_catalog.bronze.time_entries")

In [25]:
df.show()

[Stage 4:>                                                          (0 + 1) / 1]

+--------------------+-----------+------+--------------------+--------+------+--------------------+--------------------+--------------------+-----------------+-------+-------+----------+--------+--------+
|                  id|description|tagIds|              userId|billable|taskId|           projectId|         workspaceId|        timeInterval|customFieldValues|   type|kioskId|hourlyRate|costRate|isLocked|
+--------------------+-----------+------+--------------------+--------+------+--------------------+--------------------+--------------------+-----------------+-------+-------+----------+--------+--------+
|6808aa894f2e1a162...|         FN|    []|5e95c064ea8094116...|    true|  NULL|61e54e2ddc3256444...|5e95c064ea8094116...|{PT8H, 2025-04-21...|               []|REGULAR|   NULL|   {0, R$}| {0, R$}|   false|
|6808aa91762e83002...|         FN|    []|5e95c064ea8094116...|    true|  NULL|61e54e2ddc3256444...|5e95c064ea8094116...|{PT8H, 2025-04-22...|               []|REGULAR|   NULL|   {0

In [26]:
spark.read.format("iceberg").load("s3a://staging/bronze/time_entries").show()

+--------------------+-----------+------+--------------------+--------+------+--------------------+--------------------+--------------------+-----------------+-------+-------+----------+--------+--------+
|                  id|description|tagIds|              userId|billable|taskId|           projectId|         workspaceId|        timeInterval|customFieldValues|   type|kioskId|hourlyRate|costRate|isLocked|
+--------------------+-----------+------+--------------------+--------+------+--------------------+--------------------+--------------------+-----------------+-------+-------+----------+--------+--------+
|6808aa894f2e1a162...|         FN|    []|5e95c064ea8094116...|    true|  NULL|61e54e2ddc3256444...|5e95c064ea8094116...|{PT8H, 2025-04-21...|               []|REGULAR|   NULL|   {0, R$}| {0, R$}|   false|
|6808aa91762e83002...|         FN|    []|5e95c064ea8094116...|    true|  NULL|61e54e2ddc3256444...|5e95c064ea8094116...|{PT8H, 2025-04-22...|               []|REGULAR|   NULL|   {0

In [30]:
# spark.sql("SHOW NAMESPACES IN clockify_catalog").show()  # Lists all databases (namespaces)
spark.sql("SHOW TABLES IN clockify_catalog.bronze").show()  # Lists all tables in bronze

+---------+------------+-----------+
|namespace|   tableName|isTemporary|
+---------+------------+-----------+
|   bronze|time_entries|      false|
+---------+------------+-----------+

